In [1]:
from tree_manager import Tree, TreeNode, TreeEdge, init_db, SessionLocal
init_db()
session = SessionLocal()

In [2]:
tree = Tree.get(session, id=1)
new_tag = tree.create_tag(session, "release-v1.0", description="Initial stable release")

In [3]:
modified_tree = tree.create_new_tree_version_from_tag(session, "release-v1.0")
new_node = modified_tree.add_node(session, data={"setting": "new_value"})
modified_tree.create_tag(session, "release-v1.1", description="Added new setting")

In [4]:

# Adding an edge between nodes
modified_tree.add_edge(session, incoming_node_id=1, outgoing_node_id=2, data={"weight": 0.5})

# Retrieving historical configuration
old_config = tree.get_by_tag(session, "release-v1.0")

In [11]:
main_tree = tree.get_by_tag(session, "release-v1.0")
feature_branch = main_tree.create_new_tree_version_from_tag(session, "release-v1.0")

In [12]:
node1 = feature_branch.add_node(session, data={"feature_flag": True})
node2 = feature_branch.add_node(session, data={"config": "new_setting"})
feature_branch.add_edge(session, incoming_node_id=node1.id, outgoing_node_id==node2.id, data={"relation": "depends_on"})
feature_branch.create_tag(session, "feature-x-v1", description="Feature X implementation")

TypeError: add_edge() got an unexpected keyword argument 'node_id_1'